In [1]:
#Download the dataset

!gdown 1VGcLuhDo3N73T7C2mQ2KA4DxOwE-Y2b9

Downloading...
From: https://drive.google.com/uc?id=1VGcLuhDo3N73T7C2mQ2KA4DxOwE-Y2b9
To: /content/Credit_score.csv
100% 27.4M/27.4M [00:00<00:00, 80.4MB/s]


In [2]:
# libraries to analyze data
import numpy as np
import pandas as pd

# libraries to visualize data
import matplotlib.pyplot as plt
import seaborn as sns

# libraries to perform stats
import scipy.stats as stats

# **Credit Score Dataset**

## **Dataset Cleaning**

In [3]:
# read the file
cs_df = pd.read_csv('/content/Credit_score.csv', encoding='unicode_escape')
cs_df.head(5)

<ipython-input-3-1171080452e1>:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  cs_df = pd.read_csv('/content/Credit_score.csv', encoding='unicode_escape')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529544,High_spent_Small_value_payments,312.4940887
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.2802216,Low_spent_Large_value_payments,284.6291625
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.69952126,Low_spent_Medium_value_payments,331.2098629
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580744,Low_spent_Small_value_payments,223.4513097
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.42015309,High_spent_Medium_value_payments,341.489231


In [4]:
#Drop Id and SSN Attribut which are not required for our analysis
cs_df = cs_df.drop(columns=['ID', 'SSN'])

In [5]:
cs_df.head(1)

,Customer_ID,Month,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,CUS_0xd40,January,Aaron Maashoh,23,Scientist,19114.12,1824.843333,3,4,3,...,4.0,_,809.98,26.82262,22 Years and 1 Months,No,49.574949,80.41529544,High_spent_Small_value_payments,312.4940887


In [6]:
cs_df.Month.unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August'], dtype=object)

**Insight**

* We should have 8 months worth of data for each customer

In [7]:
cs_df.shape

(100000, 25)

In [8]:
cs_df.Customer_ID.nunique()

12500

**Insight**

* 12,500 unique customers with 8 rows 1 for each month is matching with 1,00,000 of dataset row size.

In [9]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Name                      90015 non-null   object 
 3   Age                       100000 non-null  object 
 4   Occupation                100000 non-null  object 
 5   Annual_Income             100000 non-null  object 
 6   Monthly_Inhand_Salary     84998 non-null   float64
 7   Num_Bank_Accounts         100000 non-null  int64  
 8   Num_Credit_Card           100000 non-null  int64  
 9   Interest_Rate             100000 non-null  int64  
 10  Num_of_Loan               100000 non-null  object 
 11  Type_of_Loan              88592 non-null   object 
 12  Delay_from_due_date       100000 non-null  int64  
 13  Num_of_Delayed_Payment    92998 non-null   ob

### **Convert Annual Income from object to float**

In [10]:
#Replace '_' and covert to float

temp_series = cs_df['Annual_Income'].str.replace('_', '')
cs_df['Annual_Income'] = round(temp_series.astype('float64'),0)
cs_df['Annual_Income'] = cs_df.groupby('Customer_ID')['Annual_Income'].transform(lambda x: (x.mode()[0]))

### **No of Bank Accounts**

### **Monthly Inhand Salary**

**Approach:**

* We are replacing the null values using the most frequent value of that particular customer **assuming** that few months the data is reported properly and missed for few months.

In [11]:
cs_df['Monthly_Inhand_Salary'] = cs_df.groupby('Customer_ID')['Monthly_Inhand_Salary'].transform(lambda x: x.fillna(x.mode()[0]))

In [12]:
cs_df['Monthly_Inhand_Salary'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100000 entries, 0 to 99999
Series name: Monthly_Inhand_Salary
Non-Null Count   Dtype  
--------------   -----  
100000 non-null  float64
dtypes: float64(1)
memory usage: 781.4 KB


### **Number of Bank Accounts, Number of Credit Cards, Interest Rate, Num of Loans**

**Approach:**

We are replacing the extreme values using the most frequent value of that particular customer assuming that few months the data is reported properly and incorrectly reported for few months.

In [13]:
cs_df['Num_Bank_Accounts'] = cs_df.groupby('Customer_ID')['Num_Bank_Accounts'].transform(lambda x: (x.mode()[0]))
cs_df['Num_Credit_Card'] = cs_df.groupby('Customer_ID')['Num_Credit_Card'].transform(lambda x: (x.mode()[0]))
cs_df['Interest_Rate'] = cs_df.groupby('Customer_ID')['Interest_Rate'].transform(lambda x: (x.mode()[0]))
cs_df['Num_of_Loan'] = cs_df.groupby('Customer_ID')['Num_of_Loan'].transform(lambda x: (x.mode()[0]))

In [14]:
cs_df['Num_Bank_Accounts'].unique()

array([ 3,  2,  1,  7,  4,  0,  8,  5,  6,  9, 10, -1])

In [15]:
#Replace -1 to 0 as -1 doesn't make sense. replace it with 0

cs_df['Num_Bank_Accounts'] = cs_df['Num_Bank_Accounts'].replace(-1, 0)

In [16]:
cs_df['Num_Bank_Accounts'].unique()

array([ 3,  2,  1,  7,  4,  0,  8,  5,  6,  9, 10])

In [17]:
cs_df['Num_Credit_Card'].unique()

array([ 4,  5,  1,  7,  6,  8,  3,  9,  2, 10, 11,  0])

In [18]:
cs_df['Interest_Rate'].unique()

array([ 3,  6,  8,  4,  5, 15,  7, 12, 20,  1, 14, 32, 16, 17, 10, 31, 25,
       18, 19,  9, 24, 13, 33, 11, 21, 29, 28, 30, 23, 34,  2, 27, 26, 22])

In [19]:
cs_df['Num_of_Loan'].unique()

array(['4', '1', '3', '0', '2', '7', '5', '6', '8', '9', '4_'],
      dtype=object)

In [20]:
#convert 4_ to 4 and convert to int64
temp_series = cs_df['Num_of_Loan'].str.replace('4_', '4')
cs_df['Num_of_Loan'] = temp_series.astype('int64')

In [21]:
cs_df['Num_of_Loan'].unique()

array([4, 1, 3, 0, 2, 7, 5, 6, 8, 9])

### **Type of Loan**

The attribute Type_of_Loan has "and" string to be replaced and fill the missing values with the mode

In [22]:
cs_df['Type_of_Loan'] = cs_df['Type_of_Loan'].str.replace(' and ', ' ')

In [23]:
cs_df['Type_of_Loan'] = cs_df['Type_of_Loan'].str.replace(', ', ',')

In [24]:
cs_df['Type_of_Loan'] = cs_df.groupby('Customer_ID')['Type_of_Loan'].transform(lambda x: x.fillna(x.mode()))

In [25]:
cs_df['Type_of_Loan'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100000 entries, 0 to 99999
Series name: Type_of_Loan
Non-Null Count  Dtype 
--------------  ----- 
88592 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


In [26]:
df_split = cs_df[['Customer_ID', 'Month', 'Type_of_Loan']]

In [27]:
# Split the 'Type_of_Loan' column into separate rows
df_split1 = df_split['Type_of_Loan'].str.split(',', expand=True).stack().reset_index(level=1, drop=True)
df_split1.name = 'Loan_Type'
df_split = df_split.drop(columns=['Type_of_Loan']).join(df_split1)
# Count each loan type for each customer
loan_counts = df_split.groupby(['Customer_ID', 'Month', 'Loan_Type']).size().unstack(fill_value=0)

In [28]:
#Merge credit score dataset and loan counts based on Customer ID and Month preserving the rows from Credit Score dataset
cs_df = pd.merge(cs_df, loan_counts, on=['Customer_ID', 'Month'], how='left')

In [29]:
#Drop Type_of_Loan as we have extracted  each loan type
cs_df = cs_df.drop(columns=['Type_of_Loan'])

In [30]:
cs_df.shape

(100000, 33)

**Insight**

The rows count remain **intact** and columns count increased.

### **Number of Delayed Payments, Changed Credit Limit, Num of Credit Inquires, Credit_Mix**

**Approach:**

We are replacing the extreme values or missing values using the most frequent value of that particular customer assuming that few months the data is reported properly and incorrectly reported for few months.

In [31]:
#Replace '_'

cs_df['Num_of_Delayed_Payment'] = cs_df['Num_of_Delayed_Payment'].str.replace('_', '')
cs_df['Changed_Credit_Limit'] = cs_df['Changed_Credit_Limit'].str.replace('_', '0')
cs_df['Credit_Mix'] = cs_df['Credit_Mix'].str.replace('_', '')

In [32]:
cs_df['Num_of_Delayed_Payment'] = cs_df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: (x.mode()[0]))
cs_df['Changed_Credit_Limit'] = cs_df.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda x: (x.mode()[0]))
cs_df['Num_Credit_Inquiries'] = cs_df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: (x.mode()[0]))
cs_df['Credit_Mix'] = cs_df.groupby('Customer_ID')['Credit_Mix'].transform(lambda x: (x.mode()[0]))

In [33]:
cs_df['Num_of_Delayed_Payment'] = cs_df['Num_of_Delayed_Payment'].astype('int64')
cs_df['Changed_Credit_Limit'] = cs_df['Changed_Credit_Limit'].astype('float64')
cs_df['Num_Credit_Inquiries'] = cs_df['Num_Credit_Inquiries'].astype('int64')

### **Outstanding Debt, Payment of Min Amount, Total EMI per month, Amount invested monthly, Monthly_Balance**

**Approach:**

We are replacing the extreme values or missing values using the most frequent value of that particular customer assuming that few months the data is reported properly and incorrectly reported for few months.

In [34]:
#Replace '_' / 'NM'

cs_df['Outstanding_Debt'] = cs_df['Outstanding_Debt'].str.replace('_', '')
cs_df['Payment_of_Min_Amount'] = cs_df['Payment_of_Min_Amount'].str.replace('NM', 'No')
cs_df['Amount_invested_monthly'] = cs_df['Amount_invested_monthly'].str.replace('_', '')

In [35]:
cs_df['Total_EMI_per_month'] = cs_df.groupby('Customer_ID')['Total_EMI_per_month'].transform(lambda x: (x.mode()[0]))
cs_df['Amount_invested_monthly'] = cs_df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: (x.mode()[0]))

In [36]:
cs_df['Monthly_Balance'] = cs_df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.fillna(x.mode()))

In [37]:
# Function to replace '__-333333333333333333333333333__' with mode at customer level
def replace_with_mode(group):
    mode_value = group[group != '__-333333333333333333333333333__'].mode()[0]
    return group.replace('__-333333333333333333333333333__', mode_value)

In [38]:
cs_df['Monthly_Balance'] = cs_df.groupby('Customer_ID')['Monthly_Balance'].transform(replace_with_mode)

In [39]:
cs_df['Outstanding_Debt'] = cs_df['Outstanding_Debt'].astype('float64')
cs_df['Amount_invested_monthly'] = cs_df['Amount_invested_monthly'].astype('float64')
cs_df['Monthly_Balance'] = cs_df['Monthly_Balance'].astype('float64')

In [40]:
cs_df['Monthly_Balance'] = cs_df['Monthly_Balance'].fillna(0)

### **Payment Behaviour**

In [41]:
# Function to replace '!@9#%8' with mode at customer level
def replace_with_mode(group):
    mode_value = group[group != '!@9#%8'].mode()[0]
    return group.replace('!@9#%8', mode_value)


In [42]:
cs_df['Payment_Behaviour'] = cs_df.groupby('Customer_ID')['Payment_Behaviour'].transform(replace_with_mode)

In [43]:
# Split the 'Payment_Behaviour' column into two new columns
cs_df[['Spent_Level', 'Value_Payments']] = cs_df['Payment_Behaviour'].str.split('_spent_', expand=True)

In [44]:
#Drop Type_of_Loan as we have extracted  each loan type
cs_df = cs_df.drop(columns=['Payment_Behaviour'])

In [45]:
cs_df.shape

(100000, 34)

**Insight**

The rows count remain **intact** and columns count increased.

In [46]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 34 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Name                      90015 non-null   object 
 3   Age                       100000 non-null  object 
 4   Occupation                100000 non-null  object 
 5   Annual_Income             100000 non-null  float64
 6   Monthly_Inhand_Salary     100000 non-null  float64
 7   Num_Bank_Accounts         100000 non-null  int64  
 8   Num_Credit_Card           100000 non-null  int64  
 9   Interest_Rate             100000 non-null  int64  
 10  Num_of_Loan               100000 non-null  int64  
 11  Delay_from_due_date       100000 non-null  int64  
 12  Num_of_Delayed_Payment    100000 non-null  int64  
 13  Changed_Credit_Limit      100000 non-null  fl

In [47]:
#Fill nan value with 0
cs_df['Auto Loan'] = cs_df['Auto Loan'].fillna(0)
cs_df['Credit-Builder Loan'] = cs_df['Credit-Builder Loan'].fillna(0)
cs_df['Debt Consolidation Loan'] = cs_df['Debt Consolidation Loan'].fillna(0)
cs_df['Home Equity Loan'] = cs_df['Home Equity Loan'].fillna(0)
cs_df['Mortgage Loan'] = cs_df['Mortgage Loan'].fillna(0)
cs_df['Payday Loan'] = cs_df['Payday Loan'].fillna(0)
cs_df['Personal Loan'] = cs_df['Personal Loan'].fillna(0)
cs_df['Student Loan'] = cs_df['Student Loan'].fillna(0)

In [48]:
#Drop Type_of_Loan Not Specified as it makes no sense
cs_df = cs_df.drop(columns=['Not Specified'])

In [49]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Name                      90015 non-null   object 
 3   Age                       100000 non-null  object 
 4   Occupation                100000 non-null  object 
 5   Annual_Income             100000 non-null  float64
 6   Monthly_Inhand_Salary     100000 non-null  float64
 7   Num_Bank_Accounts         100000 non-null  int64  
 8   Num_Credit_Card           100000 non-null  int64  
 9   Interest_Rate             100000 non-null  int64  
 10  Num_of_Loan               100000 non-null  int64  
 11  Delay_from_due_date       100000 non-null  int64  
 12  Num_of_Delayed_Payment    100000 non-null  int64  
 13  Changed_Credit_Limit      100000 non-null  fl

In [50]:
cs_df['Value_Payments'] = cs_df['Value_Payments'].str.replace('_value_payments', '')

In [51]:
cs_df.head()

,Customer_ID,Month,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,...,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Payday Loan,Personal Loan,Student Loan,Spent_Level,Value_Payments
0,CUS_0xd40,January,Aaron Maashoh,23,Scientist,19114.0,1824.843333,3,4,3,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,High,Small
1,CUS_0xd40,February,Aaron Maashoh,23,Scientist,19114.0,1824.843333,3,4,3,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Low,Large
2,CUS_0xd40,March,Aaron Maashoh,-500,Scientist,19114.0,1824.843333,3,4,3,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Low,Medium
3,CUS_0xd40,April,Aaron Maashoh,23,Scientist,19114.0,1824.843333,3,4,3,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Low,Small
4,CUS_0xd40,May,Aaron Maashoh,23,Scientist,19114.0,1824.843333,3,4,3,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,High,Medium


### **Credit History Age**

We will extract only the no of years and if we consider the maximum value for each customer

In [52]:
# Extract the numeric part before 'Years' and convert to numeric, handling 'NA' values
cs_df['Credit_History_Years'] = cs_df['Credit_History_Age'].str.extract(r'(\d+)').astype(float)

# Group by 'Customer_ID' and find the maximum 'Years' for each customer
cs_df['Credit_History_Years'] = cs_df.groupby('Customer_ID')['Credit_History_Years'].transform(lambda x: (x.max()))

In [53]:
cs_df = cs_df.drop(columns=['Credit_History_Age'])

In [54]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Name                      90015 non-null   object 
 3   Age                       100000 non-null  object 
 4   Occupation                100000 non-null  object 
 5   Annual_Income             100000 non-null  float64
 6   Monthly_Inhand_Salary     100000 non-null  float64
 7   Num_Bank_Accounts         100000 non-null  int64  
 8   Num_Credit_Card           100000 non-null  int64  
 9   Interest_Rate             100000 non-null  int64  
 10  Num_of_Loan               100000 non-null  int64  
 11  Delay_from_due_date       100000 non-null  int64  
 12  Num_of_Delayed_Payment    100000 non-null  int64  
 13  Changed_Credit_Limit      100000 non-null  fl

In [55]:
#Drop Name column as well as it doesn't add any value to credit worthiness
cs_df = cs_df.drop(columns=['Name'])

### **Age, Occupation**

In [56]:
#Replace '_'

cs_df['Age'] = cs_df['Age'].str.replace('_', '')

In [57]:
cs_df['Age'] = cs_df.groupby('Customer_ID')['Age'].transform(lambda x: (x.mode()[0]))
cs_df['Occupation'] = cs_df.groupby('Customer_ID')['Occupation'].transform(lambda x: (x.mode()[0]))

In [58]:
cs_df['Age'] = cs_df['Age'].astype("int64")

In [59]:
cs_df['Occupation'] = cs_df['Occupation'].str.replace('_______',"Not Available")

In [60]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Age                       100000 non-null  int64  
 3   Occupation                100000 non-null  object 
 4   Annual_Income             100000 non-null  float64
 5   Monthly_Inhand_Salary     100000 non-null  float64
 6   Num_Bank_Accounts         100000 non-null  int64  
 7   Num_Credit_Card           100000 non-null  int64  
 8   Interest_Rate             100000 non-null  int64  
 9   Num_of_Loan               100000 non-null  int64  
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    100000 non-null  int64  
 12  Changed_Credit_Limit      100000 non-null  float64
 13  Num_Credit_Inquiries      100000 non-null  in

In [61]:
cs_df.head(5)

,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Payday Loan,Personal Loan,Student Loan,Spent_Level,Value_Payments,Credit_History_Years
0,CUS_0xd40,January,23,Scientist,19114.0,1824.843333,3,4,3,4,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,High,Small,22.0
1,CUS_0xd40,February,23,Scientist,19114.0,1824.843333,3,4,3,4,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Low,Large,22.0
2,CUS_0xd40,March,23,Scientist,19114.0,1824.843333,3,4,3,4,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Low,Medium,22.0
3,CUS_0xd40,April,23,Scientist,19114.0,1824.843333,3,4,3,4,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Low,Small,22.0
4,CUS_0xd40,May,23,Scientist,19114.0,1824.843333,3,4,3,4,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,High,Medium,22.0


In [62]:
# #Save the cleaned dataset to drive

cs_df.to_csv('cleaned_credit_score.csv', index=False)

# *********   Data Cleaning is Completed *************